# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [ ]:
! pip install -U langchain

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [1]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

bio_template = """你是一位很棒的生物老师。你擅长回答生物问题。
你擅长以简洁易懂的方式回答关于物理的问题。

这是一个问题：
{input}"""

computer_template = """你是一位很棒的计算机老师。你擅长回答计算机问题。
你擅长详细回答计算机方面的问题。

这是一个问题：
{input}"""

yw_template = """你是一位很棒的汉语言文学老师。你擅长回答汉语言文学问题。
你擅长详细回答汉语言文学的问题。

这是一个问题：
{input}"""

In [3]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": bio_template,
    }, {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    }, {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": yw_template,
    },
]

In [ ]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [5]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

C:\Users\ouda2\Desktop\work-space\langchainqs\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
C:\Users\ouda2\Desktop\work-space\langchainqs\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [6]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [7]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [8]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [9]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [10]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [11]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [12]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<

In [13]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [14]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？'}
> Finished chain.
{'input': '黑体辐射是什么？', 'text': '黑体辐射是指一种理想化物体（称为黑体）在热平衡状态下发出的电磁辐射。黑体是一个完全吸收所有入射电磁辐射的物体，不反射或透射任何辐射。也就是说，黑体无论在什么波长上都具有最大吸收率。\n\n黑体辐射的特性由普朗克辐射定律描述，它表明黑体在不同温度下发出的辐射光谱。随着温度的升高，黑体辐射的峰值波长会向更短的波长（更高的频率）移动，这就是所谓的“维恩位移定律”。此外，黑体辐射的总能量输出与温度的四次方成正比，这就是“斯特藩-玻尔兹曼定律”。\n\n黑体辐射的研究在19世纪末和20世纪初对量子力学的发展起到了至关重要的作用，特别是马克斯·普朗克提出的量子假设，为解决经典物理无法解释的紫外灾变问题提供'}


In [15]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '要找到大于40的第一个质数，使得这个质数加一能被3整除，我们可以按照以下步骤进行：\n\n1. **找出大于40的质数**：\n   质数是指只能被1和自身整除的自然数。我们从41开始依次检查每个数是否是质数。\n\n2. **检查质数加一是否能被3整除**：\n   对于找到的每一个质数，我们检查质数加一是否能被3整除（即质数加一后模3结果为0）。\n\n接下来我们一步一步来：\n\n1. 检查41是否是质数：\n   - 41不能被2整除（不是偶数）。\n   - 41不能被3整除（41除以3约等于13.67，不是整数）。\n   - 41不能被5整除（结尾不是0或5）。\n\n   因此，41是质数。\n\n2. 检查41加一是否能被3整除：\n   - 41 + 1 = 42\n   - 42 ÷ 3 = 14（是整数）\n\n   因此，41加一'}


In [16]:
print(
    chain.invoke(
        "什么是蛋白质？"
    )
)



> Entering new MultiPromptChain chain...
生物: {'input': '什么是蛋白质？'}
> Finished chain.
{'input': '什么是蛋白质？', 'text': '蛋白质是由氨基酸组成的生物大分子。它们在细胞中扮演着许多重要角色，包括构建和修复组织、催化化学反应（作为酶）、运输分子（如血红蛋白运输氧气）以及调节生理过程（如激素功能）。蛋白质的结构和功能由其氨基酸序列决定，这些序列是由基因编码的。蛋白质在我们的饮食中也是重要的营养成分，主要来源包括肉类、鱼类、蛋类、奶制品和某些植物如豆类。'}


In [17]:
print(
    chain.invoke(
        "计算机由什么组成？"
    )
)



> Entering new MultiPromptChain chain...
计算机: {'input': '计算机由什么组成？'}
> Finished chain.
{'input': '计算机由什么组成？', 'text': '计算机是一种复杂的机器，由多个硬件和软件组件协同工作来执行各种任务。具体来说，计算机主要由以下几个核心部分组成：\n\n### 1. **硬件部分**\n\n#### a. **中央处理器（CPU）**\n- **功能**：CPU是计算机的“大脑”，负责执行计算机程序中的指令。它进行算术运算、逻辑运算、控制和输入/输出（I/O）操作。\n- **组成**：通常由控制单元（CU）、算术逻辑单元（ALU）和寄存器组成。\n\n#### b. **内存（RAM）**\n- **功能**：随机存取存储器（RAM）用于临时存储正在使用的数据和程序。它是易失性存储器，计算机关机时数据会丢失。\n- **类型**：动态随机存取存储器（DRAM）和静态随机存取存储器（SRAM）。\n\n#### c. **存储设备**\n- **功能**：用于长时间存储数据，即使计算机关机数据也不会丢失。\n- **类型**：硬盘驱动器（HDD'}


In [18]:
print(
    chain.invoke(
        "雷雨讲的是一个怎样的故事？"
    )
)



> Entering new MultiPromptChain chain...
汉语言文学: {'input': '雷雨讲的是一个怎样的故事？'}
> Finished chain.
{'input': '雷雨讲的是一个怎样的故事？', 'text': '《雷雨》是中国现代文学家曹禺创作的一部经典话剧，首次发表于1933年。它是一部四幕话剧，通过错综复杂的家庭关系和人物命运，揭示了20世纪初中国社会的种种矛盾和悲剧。\n\n### 故事背景\n故事发生在一个富裕的家庭——周家，时间跨度从19世纪末到20世纪初，主要集中在一天之内。周家是一个典型的封建大家庭，表面上富丽堂皇，实则暗藏各种矛盾和罪恶。\n\n### 主要人物\n- **周朴园**：周家的家长，一个富有而冷酷的资本家。\n- **鲁侍萍**：周朴园的前妻，现为贫困的寡妇。\n- **周蘩漪**：周朴园的现任妻子，性格压抑但内心充满反抗。\n- **周萍**：周朴园与鲁侍萍的儿子，周家的长子，性格复杂矛盾。\n- **周冲**：'}
